# Projeto Python IA: Inteligência Artificial e Previsões

### Case: Score de Crédito dos dados_clientes

Você foi contratado por um banco para conseguir definir o score de crédito dos dados_clientes. Você precisa analisar todos os clientes do banco e, com base nessa análise, criar um modelo que consiga ler as informações do cliente e dizer automaticamente o score de crédito dele: Ruim, Ok, Bom

In [19]:
# bibliotecas
import pandas as pd
from sklearn.preprocessing   import LabelEncoder           # transformas as informações em texto (objetc) em número (int)
from sklearn.model_selection import train_test_split       # separa os dados para treino e para teste
from sklearn.ensemble        import RandomForestClassifier # IA de modelo de árvore de decisão
from sklearn.metrics         import accuracy_score         # calcula a previsão de acerto
from scipy.stats             import norm                   # mostra o valor em porcentagem


# 1. importar a basse de dados dos dados_clientes
dados_clientes = pd.read_csv('clientes.csv')

display(dados_clientes)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.465380,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.465380,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.465380,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.465380,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.465380,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,37932,4,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,23.0,...,378.0,24.028477,alto_gasto_pagamento_alto,479.866228,Poor,1,0,0,0,1
99996,37932,5,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,18.0,...,379.0,24.028477,alto_gasto_pagamento_medio,496.651610,Poor,1,0,0,0,1
99997,37932,6,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,27.0,...,380.0,24.028477,alto_gasto_pagamento_alto,516.809083,Poor,1,0,0,0,1
99998,37932,7,25.0,mecanico,39628.99,4.0,6.0,7.0,2.0,20.0,...,381.0,24.028477,baixo_gasto_pagamento_alto,319.164979,Standard,1,0,0,0,1


In [20]:
# 2. analisar se o arquivo possui alguma informação vazia
display(dados_clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

None

In [21]:
# 3. transformar as colunas de object para int
codificador = LabelEncoder()

# transformar todas as colunas em int, menos a 'score_credito' pois é a previsão a ser feita
for coluna in dados_clientes.columns:
    if (dados_clientes[coluna].dtype == 'object' and coluna != 'score_credito'):
        dados_clientes[coluna] = codificador.fit_transform(dados_clientes[coluna])

# verificar se ocorreu a transformação das colunas corretamente
display(dados_clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  int32  
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  int32  
 13  divida_total              100000 non-null  fl

None

In [25]:
# 4. separar as colunas que iremos listar para treinar a IA

# x são as colunas que vamos utilizar para o prever o score_credito 
# y coluna que queremos que a IA calcule

x = dados_clientes.drop(['id_cliente', 'score_credito'], axis=1) # removendo as colunas 'id_cliente' e 'score_credito' pois não serão usadas no x
y = dados_clientes['score_credito'] # coluna que a IA irá usar para prever o 'socre_credito'


# dividir os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=1) # 30% dos dados estão sendo destinados para teste


# 5. treinar a IA para ler o dados de treino
IA_arvore_decisao = RandomForestClassifier()
IA_arvore_decisao.fit(x_treino, y_treino) 


# 6. calcular a previsão de acerto com os dados de teste
previsao = IA_arvore_decisao.predict(x_teste)
porcentagem_acerto = accuracy_score(y_teste, previsao)

display(round(norm.cdf(porcentagem_acerto) * 100, 2))

# previsão de acerto de 79,56%

79.56

In [29]:
# 7. fazer novas previsões com outros dados
dados_novos_clientes = pd.read_csv('novos_clientes.csv')

# transformar as colunas de 'object' para 'int' (deixando no modo padrão para IA)
for coluna in dados_novos_clientes.columns:
    if (dados_novos_clientes[coluna].dtype == 'object' and coluna != 'score_credito'):
        dados_novos_clientes[coluna] = codificador.fit_transform(dados_novos_clientes[coluna])

# realizar a previsão do 'score de crédito'
previsao_novos_clintes = IA_arvore_decisao.predict(dados_novos_clientes)
display(dados_novos_clientes)
display(previsao_novos_clintes)

,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,num_pagamentos_atrasados,...,taxa_uso_credito,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,1,31.0,1,19300.340,6.0,7.0,17.0,5.0,52.0,19.0,...,29.934186,218.0,44.50951,1,312.487689,1,1,0,0,0
1,4,32.0,0,12600.445,5.0,5.0,10.0,3.0,25.0,18.0,...,28.819407,12.0,0.00000,2,300.994163,0,0,0,0,1
2,2,48.0,1,20787.690,8.0,6.0,14.0,7.0,24.0,14.0,...,34.235853,215.0,0.00000,0,345.081577,0,1,0,1,0


array(['Poor', 'Good', 'Standard'], dtype=object)

In [38]:
# 8. analisar quais são as colunas que são as mais importante para o score de crédito
colunas = list(x_teste.columns)

importancia = pd.DataFrame(index=colunas, data=IA_arvore_decisao.feature_importances_)
importancia = round(importancia * 100)

display(importancia)

# tópicos mais importantes para o score de crédito:
# 1° divida total
# 2° idade historico de credito
# 3° juros emprestimo
# 4° dias atraso

,0
mes,4.0
idade,4.0
profissao,3.0
salario_anual,5.0
num_contas,3.0
num_cartoes,4.0
juros_emprestimo,8.0
num_emprestimos,3.0
dias_atraso,7.0
num_pagamentos_atrasados,4.0
